# Метод предобработки датасета

In [292]:
import shutil
import os

In [234]:
# shutil.rmtree('/kaggle/working/Expenses/')
# os.mkdir('/kaggle/working/Expenses/')

In [295]:
def make_expanses(path, file_name):
    df = pd.DataFrame(pd.read_excel(path))
    
    df.fillna(0, inplace=True)
    
    cols = ['name', 'year', 'smeta', 'place', 'code', 'code_type', 'def', 'definition', 'firstly', 'given', 'given_end', 'date']
    
    df.columns = cols
    
    df = df[(df.name == 0) & (df.year != 0) & (df.firstly == 0)]
    
    df = df.drop(columns = ['name', 'code', 'code_type', 'firstly', 'smeta', 'place'],axis = 1)
# для сохранения обработанных csv
#     df.to_csv('/kaggle/working/Expenses/' + file_name)
    
    return df

In [296]:
expanses_df = pd.DataFrame({'year':[0],'definition': [0], 'def':[0], 'given': [0], 'given_end': [0], 'date': [0]})

import os
for dirname, _, filenames in os.walk('/kaggle/input/digital-breakthrough/train_data 2/train_data 2/Expenses'):
    for filename in filenames:
        expanse = make_expanses(os.path.join(dirname, filename), filename.replace('xlsx', 'csv'))
        expanses_df = pd.concat([expanses_df, expanse])

expanses_df = expanses_df[expanses_df.year != 0]
expanses_df.to_csv('Expenses.csv')
expanses_df

,year,definition,def,given,given_end,date
25,2020,134.1100.010000 Плата за технологическое присо...,Расходы по оплате потребления электроэнергии,32790.96,32790.96,31.12.2020
28,2020,Оплата за электроэнергию по тарифам зд. Вл-А...,Расходы по оплате потребления электроэнергии,2274.13,2274.13,31.12.2020
30,2020,Оплата за электроэнергию по тарифам зд. ул.Све...,Расходы по оплате потребления электроэнергии,238482.63,238482.63,31.12.2020
32,2020,Оплата за электроэнергию по тарифам зд. г.Нахо...,Расходы по оплате потребления электроэнергии,614813.95,614813.95,31.12.2020
34,2020,Оплата за электроэнергию по тарифам зд. Склад...,Расходы по оплате потребления электроэнергии,22413.58,22413.58,31.12.2020
...,...,...,...,...,...,...
3549,2019,Оплата потребления газа по тарифам зд. ул.Киро...,Расходы на коммунальные услуги,609.08,609.08,18.09.2019
3550,2019,Оплата потребления газа по тарифам зд. ул.Киро...,Расходы на коммунальные услуги,26503.15,26503.15,14.10.2019
3551,2019,Оплата потребления газа по тарифам зд. ул.Киро...,Расходы на коммунальные услуги,59959.81,59959.81,12.11.2019
3552,2019,Оплата потребления газа по тарифам зд. ул.Киро...,Расходы на коммунальные услуги,105621.11,105621.11,16.12.2019


In [304]:
import re

def extractStreetAndNumber(text):
    # Регулярное выражение для названия улицы и номера дома
    regex = r'(?:ул\.|пер\.|улица|Улица|ул\.|зд\.|ул|проспект|пр\.|пр-т|пр-кт|переулок|Переулок|пер\.|шоссе|ш\.|пл\.|площадь|Пр-т|Проспект|м-н|просп\.|пл\.|ш\.|пр\.|г\.|зд\.)\s*([А-Яа-яЁё\s-]+),?\s*(\d+[а-яА-Яa-zA-Z]?)'
    matches = re.search(regex, text)
    
    if 'Океанский пр-т, 34' in text:
        return 'Океанский', '34'
    
    if 'РКЦ Уссурийск' in text:
        return 'РКЦ Уссурийск' 
    
    if 'ул.Колхозная 32б' in text:
        return 'Колхозная', '32б'
    
    if 'ул. Дежнева,7' in text:
        return 'ул. Дежнева', '7'
    
    if 'М.Амурского, 42' in text:
        return 'М.Амурского', '42'

    if matches:
        street = matches.group(1).strip()
        number = matches.group(2).strip()
        return street, number
    else:
        return None, None

In [305]:
# Применение функции к колонке definition
expanses_df[['street', 'house_number']] = expanses_df['definition'].apply(
    lambda x: pd.Series(extractStreetAndNumber(x))
)

# Подсчет уникальных значений
unique_streets = expanses_df['street'].nunique()
unique_numbers = expanses_df['house_number'].nunique()

print(f"Уникальных улиц: {unique_streets}, Уникальных номеров домов: {unique_numbers}")
expanses_df['street']

Уникальных улиц: 74, Уникальных номеров домов: 81


25         Калинина
28           Седова
30      Светланская
32             None
34        Фонтанная
           ...     
3549         Кирова
3550         Кирова
3551         Кирова
3552         Кирова
3555           None
Name: street, Length: 23150, dtype: object

In [306]:
# Объединение колонок 'street' и 'house_number' в одну колонку 'street'
expanses_df['street'] = expanses_df['street'].fillna('') + ' ' + expanses_df['house_number'].fillna('')
expanses_df['street'] = expanses_df['street'].str.strip()

# Удаление колонки 'house_number', так как ее значение уже включено в 'street'
expanses_df.drop(columns=['house_number'], inplace=True)

In [308]:
expanses_df = expanses_df.dropna(subset=['street'])

In [309]:
expanses_df = expanses_df.drop(columns=['definition'])

In [310]:
expanses_df

,year,def,given,given_end,date,street
25,2020,Расходы по оплате потребления электроэнергии,32790.96,32790.96,31.12.2020,Калинина 261
28,2020,Расходы по оплате потребления электроэнергии,2274.13,2274.13,31.12.2020,Седова 23
30,2020,Расходы по оплате потребления электроэнергии,238482.63,238482.63,31.12.2020,Светланская 111
32,2020,Расходы по оплате потребления электроэнергии,614813.95,614813.95,31.12.2020,
34,2020,Расходы по оплате потребления электроэнергии,22413.58,22413.58,31.12.2020,Фонтанная 19
...,...,...,...,...,...,...
3549,2019,Расходы на коммунальные услуги,609.08,609.08,18.09.2019,Кирова 17
3550,2019,Расходы на коммунальные услуги,26503.15,26503.15,14.10.2019,Кирова 17
3551,2019,Расходы на коммунальные услуги,59959.81,59959.81,12.11.2019,Кирова 17
3552,2019,Расходы на коммунальные услуги,105621.11,105621.11,16.12.2019,Кирова 17


In [311]:
unique_defininitions = expanses_df['def'].unique()
unique_defininitions


array(['Расходы по оплате потребления электроэнергии',
       'Расходы по оплате всех видов отопления зданий и сооружений (кроме электро- и газового снабжения)',
       'Расходы по оплате услуг водоснабжения, водоотведения',
       'Расходы на коммунальные услуги (расходы прошлых лет)',
       'Расходы по оплате потребления газа',
       'Оплата потребления электроэнергии',
       'Оплата потребления тепловой энергии',
       'Оплата водоснабжения, канализации', 'Оплата потребления газа',
       'Расходы на коммунальные услуги',
       'Оплата всех видов отопления зданий и сооружений (кроме электро- и газового снабжения)',
       'Оплата услуг водоснабжения, водоотведения',
       'Коммунальные услуги (расходы прошлых лет)'], dtype=object)

In [312]:
def make_labels(col):
    nums_definitions = list(range(1, len(col.unique())+1))
    dict_definiti = dict(zip(col.unique(), nums_definitions))
    return dict_definiti

In [313]:
dict_label = make_labels(expanses_df['def'])
dict_label

{'Расходы по оплате потребления электроэнергии': 1,
 'Расходы по оплате всех видов отопления зданий и сооружений (кроме электро- и газового снабжения)': 2,
 'Расходы по оплате услуг водоснабжения, водоотведения': 3,
 'Расходы по оплате потребления газа': 4,
 'Оплата потребления электроэнергии': 5,
 'Оплата потребления тепловой энергии': 6,
 'Оплата водоснабжения, канализации': 7,
 'Расходы на коммунальные услуги': 8,
 'Оплата всех видов отопления зданий и сооружений (кроме электро- и газового снабжения)': 9,
 'Оплата услуг водоснабжения, водоотведения': 10}

In [314]:
expanses_df["def"] = expanses_df["def"].map(dict_label).fillna(np.nan)
expanses_df.dropna(inplace=True)
expanses_df

,year,def,given,given_end,date,street
25,2020,1.0,32790.96,32790.96,31.12.2020,Калинина 261
28,2020,1.0,2274.13,2274.13,31.12.2020,Седова 23
30,2020,1.0,238482.63,238482.63,31.12.2020,Светланская 111
32,2020,1.0,614813.95,614813.95,31.12.2020,
34,2020,1.0,22413.58,22413.58,31.12.2020,Фонтанная 19
...,...,...,...,...,...,...
3549,2019,8.0,609.08,609.08,18.09.2019,Кирова 17
3550,2019,8.0,26503.15,26503.15,14.10.2019,Кирова 17
3551,2019,8.0,59959.81,59959.81,12.11.2019,Кирова 17
3552,2019,8.0,105621.11,105621.11,16.12.2019,Кирова 17


In [315]:
expanses_df['date'] = pd.to_datetime(expanses_df['date'], format='%d.%m.%Y').dt.month
expanses_df

,year,def,given,given_end,date,street
25,2020,1.0,32790.96,32790.96,12,Калинина 261
28,2020,1.0,2274.13,2274.13,12,Седова 23
30,2020,1.0,238482.63,238482.63,12,Светланская 111
32,2020,1.0,614813.95,614813.95,12,
34,2020,1.0,22413.58,22413.58,12,Фонтанная 19
...,...,...,...,...,...,...
3549,2019,8.0,609.08,609.08,9,Кирова 17
3550,2019,8.0,26503.15,26503.15,10,Кирова 17
3551,2019,8.0,59959.81,59959.81,11,Кирова 17
3552,2019,8.0,105621.11,105621.11,12,Кирова 17


In [316]:
dict_streets = make_labels(expanses_df['street'])
dict_streets


{'Калинина 261': 1,
 'Седова 23': 2,
 'Светланская 111': 3,
 'Фонтанная 19': 4,
 'Семеновская 29а': 5,
 'Мордовцева 6': 6,
 'Светланская 69': 7,
 'Светланская 71': 8,
 'Светланская 73': 9,
 'Светланская 73А': 10,
 'Океанский пр-т 34': 11,
 'Пушкинская 8Б': 12,
 'Некрасова 102': 13,
 'Пушкина 59': 14,
 'Ленина 64': 15,
 'Пушкинская 8б': 16,
 'пр-т 50': 17,
 'Горького 7': 18,
 'Дежнева 7': 19,
 'Колхозная 32': 20,
 '60': 21,
 'Чапаева 5А': 22,
 'Святителя Иннокентия 17': 23,
 'Богдана Хмельницкого 52': 24,
 'Жуковский 15': 25,
 'Ленина 29': 26,
 'Профсоюзная 12': 27,
 'Пушкина 4': 28,
 'Пушкина 6': 29,
 'Поротова 14': 30,
 'Ленинская 33': 31,
 'Партизанская 11': 32,
 'Ленинская 22': 33,
 'Карла Маркса 29': 34,
 'Первостроителей 19': 35,
 'Советская 26': 36,
 'Воронежская 154': 37,
 'М-Амурского 42': 38,
 'М-Амурского 21': 39,
 'Калинина 95': 40,
 'М- Амурского 21': 41,
 'М- Амурского 42': 42,
 'Советская 27': 43,
 'Кирова 17': 44,
 'Ленина 70': 45,
 'Ленина 22': 46,
 'Верхнепортовая 44':

In [317]:
expanses_df["street"] = expanses_df["street"].map(dict_streets).fillna(np.nan)
expanses_df.dropna(inplace=True)
expanses_df

,year,def,given,given_end,date,street
25,2020,1.0,32790.96,32790.96,12,1.0
28,2020,1.0,2274.13,2274.13,12,2.0
30,2020,1.0,238482.63,238482.63,12,3.0
34,2020,1.0,22413.58,22413.58,12,4.0
36,2020,1.0,4397.3,4397.3,12,5.0
...,...,...,...,...,...,...
3548,2019,8.0,3195.06,3195.06,9,44.0
3549,2019,8.0,609.08,609.08,9,44.0
3550,2019,8.0,26503.15,26503.15,10,44.0
3551,2019,8.0,59959.81,59959.81,11,44.0


In [318]:
# res = (expanses_df
#        .assign(n=expanses_df.groupby(['street', 'year']).cumcount())
#        .pivot_table(index="n", columns=['street', 'year', 'date'], values=['given', 'given_end',], fill_value=0)
#        )

res = (expanses_df
       .assign(n=expanses_df.groupby(['street', 'year']).cumcount())
       .pivot_table(columns=['date'], values=['given'], index=['street', 'year', 'def'], fill_value=0)
       )

In [319]:
res

given                                                  \
date                      1          2              3         4           5    
street year def                                                                
1.0    2016 5.0        0.000   21330.22   24348.710000  21696.45  25386.7800   
            6.0        0.000   54133.09   47974.530000  40132.41  26865.6300   
            7.0      538.230     708.20     736.526667    358.96    882.8175   
       2017 5.0     8141.060   30356.60    5968.230000  13546.67  26469.2800   
            6.0    51325.470   52161.40   53143.920000  39668.11  31544.6300   
...                      ...        ...            ...       ...         ...   
132.0  2021 5.0    87810.720   83238.14   81613.100000  85691.08  85958.6600   
            9.0   169220.040  127554.81  105461.900000  87884.96  54349.8700   
            10.0     536.976     455.78     455.780000    455.78    455.7800   
133.0  2021 9.0        0.000    6292.82    6292.820000   6292.82      0.0000   
134.0  2019 8.0      417.700      55.72       0.000000     43.16      0.0000   

                                                                          \
date                        6             7             8             9    
street year def                                                            
1.0    2016 5.0   22238.080000   8115.533333  18901.520000  27978.350000   
            6.0    5259.580000  -3482.250000      0.000000      0.000000   
            7.0     963.146667    -15.072500   1335.146667   1243.923333   
       2017 5.0   26992.810000  24477.850000  34989.730000  29371.930000   
            6.0    4511.850000      0.000000      0.000000      0.000000   
...                        ...           ...           ...           ...   
132.0  2021 5.0   89589.320000  65937.980000  69230.760000  70241.770000   
            9.0   27601.665000    712.940000    748.330000  -5897.460000   
            10.0    455.780000    455.780000    512.495000   1917.695000   
133.0  2021 9.0       0.000000      0.000000      0.000000      0.000000   
134.0  2019 8.0       0.000000      0.000000      0.000000      0.000000   

                                                              
date                        10             11             12  
street year def                                               
1.0    2016 5.0   24070.430000   26939.060000   32267.665000  
            6.0       0.000000       0.000000   50795.095000  
            7.0     733.596667     861.173333     813.251667  
       2017 5.0   28638.720000   32199.540000   15747.605000  
            6.0       0.000000    5158.260000   23606.515000  
...                        ...            ...            ...  
132.0  2021 5.0   71432.230000   81697.990000   92885.100000  
            9.0   25790.070000  127876.280000  206253.040000  
            10.0    591.106667     591.106667     591.106667  
133.0  2021 9.0       0.000000       0.000000       0.000000  
134.0  2019 8.0       0.000000       0.000000       0.000000  

[662 rows x 12 columns]

In [320]:
res.to_csv('els.csv')
res

given                                                  \
date                      1          2              3         4           5    
street year def                                                                
1.0    2016 5.0        0.000   21330.22   24348.710000  21696.45  25386.7800   
            6.0        0.000   54133.09   47974.530000  40132.41  26865.6300   
            7.0      538.230     708.20     736.526667    358.96    882.8175   
       2017 5.0     8141.060   30356.60    5968.230000  13546.67  26469.2800   
            6.0    51325.470   52161.40   53143.920000  39668.11  31544.6300   
...                      ...        ...            ...       ...         ...   
132.0  2021 5.0    87810.720   83238.14   81613.100000  85691.08  85958.6600   
            9.0   169220.040  127554.81  105461.900000  87884.96  54349.8700   
            10.0     536.976     455.78     455.780000    455.78    455.7800   
133.0  2021 9.0        0.000    6292.82    6292.820000   6292.82      0.0000   
134.0  2019 8.0      417.700      55.72       0.000000     43.16      0.0000   

                                                                          \
date                        6             7             8             9    
street year def                                                            
1.0    2016 5.0   22238.080000   8115.533333  18901.520000  27978.350000   
            6.0    5259.580000  -3482.250000      0.000000      0.000000   
            7.0     963.146667    -15.072500   1335.146667   1243.923333   
       2017 5.0   26992.810000  24477.850000  34989.730000  29371.930000   
            6.0    4511.850000      0.000000      0.000000      0.000000   
...                        ...           ...           ...           ...   
132.0  2021 5.0   89589.320000  65937.980000  69230.760000  70241.770000   
            9.0   27601.665000    712.940000    748.330000  -5897.460000   
            10.0    455.780000    455.780000    512.495000   1917.695000   
133.0  2021 9.0       0.000000      0.000000      0.000000      0.000000   
134.0  2019 8.0       0.000000      0.000000      0.000000      0.000000   

                                                              
date                        10             11             12  
street year def                                               
1.0    2016 5.0   24070.430000   26939.060000   32267.665000  
            6.0       0.000000       0.000000   50795.095000  
            7.0     733.596667     861.173333     813.251667  
       2017 5.0   28638.720000   32199.540000   15747.605000  
            6.0       0.000000    5158.260000   23606.515000  
...                        ...            ...            ...  
132.0  2021 5.0   71432.230000   81697.990000   92885.100000  
            9.0   25790.070000  127876.280000  206253.040000  
            10.0    591.106667     591.106667     591.106667  
133.0  2021 9.0       0.000000       0.000000       0.000000  
134.0  2019 8.0       0.000000       0.000000       0.000000  

[662 rows x 12 columns]

# Обучение модели

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Подготовка данных
# Выбираем 'year', 'def', и 'street' в качестве признаков (X)
given_columns = [col for col in df.columns if 'given' in col]
df['average_given'] = df[given_columns].mean(axis=1)

X = df[['street', 'year', 'def']]
y = df['average_given']

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Обучение модели линейной регрессии
model = LinearRegression()
model.fit(X_train, y_train)

# Предсказания и оценка модели
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print("RMSE: ", rmse)
print("Пример предсказания: ", model.predict([[1, 2024, 5]])[0])  # Пример для street 1, year 2024, def 5

# Cвертка данных

In [291]:
!pip install xlrd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 2.3 MB/s eta 0:00:00a 0:00:01


In [384]:
input_file = pd.DataFrame(pd.read_excel('/kaggle/input/hackathon/file_test.xlsx'))
input_file

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,Наименование вида расходов,Направление расходов,Адрес,Дата платежа,Отнесено
1,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,Океанский 34,02:2022,NaN
2,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,Океанский 34,03:2022,NaN
3,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,Океанский 34,04:2022,NaN
4,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,Океанский 34,05:2022,NaN
...,...,...,...,...,...
76,Отопление,134.1000.02 Столовая ХБ Оплата за отопление по...,М.Амурского 42,08:2022,NaN
77,Отопление,134.1000.02 Столовая ХБ Оплата за отопление по...,М.Амурского 42,09:2022,NaN
78,Отопление,134.1000.02 Столовая ХБ Оплата за отопление по...,М.Амурского 42,10:2022,NaN
79,Отопление,134.1000.02 Столовая ХБ Оплата за отопление по...,М.Амурского 42,11:2022,NaN


In [385]:
def fix_cols(input_file):
    input_file.fillna(0, inplace=True)

    cols = ['def', 'definition', 'street', 'year', 'given']

    input_file.columns = cols

    input_file = input_file.drop(index=0 ,axis = 1)

    return input_file

In [387]:
input_file = fix_cols(input_file)

KeyError: '[0] not found in axis'

In [390]:
input_file['year'] = pd.to_datetime(input_file['year'], format='%m:%Y').dt.month
input_file

,def,definition,street,year,given
1,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,Океанский 34,2,0
2,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,Океанский 34,3,0
3,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,Океанский 34,4,0
4,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,Океанский 34,5,0
5,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,Океанский 34,6,0
...,...,...,...,...,...
76,Отопление,134.1000.02 Столовая ХБ Оплата за отопление по...,М.Амурского 42,8,0
77,Отопление,134.1000.02 Столовая ХБ Оплата за отопление по...,М.Амурского 42,9,0
78,Отопление,134.1000.02 Столовая ХБ Оплата за отопление по...,М.Амурского 42,10,0
79,Отопление,134.1000.02 Столовая ХБ Оплата за отопление по...,М.Амурского 42,11,0


In [391]:
dicti = {'Электроэнергия': 1,
 'Коммунальные услуги (расходы прошлых лет)': 8,
 'Отопление': 2}


In [392]:
input_file["def"] = input_file["def"].map(dicti).fillna(np.nan)
input_file

,def,definition,street,year,given
1,1,134.1000.01 ДГУ. Оплата за электроэнергию по т...,Океанский 34,2,0
2,1,134.1000.01 ДГУ. Оплата за электроэнергию по т...,Океанский 34,3,0
3,1,134.1000.01 ДГУ. Оплата за электроэнергию по т...,Океанский 34,4,0
4,1,134.1000.01 ДГУ. Оплата за электроэнергию по т...,Океанский 34,5,0
5,1,134.1000.01 ДГУ. Оплата за электроэнергию по т...,Океанский 34,6,0
...,...,...,...,...,...
76,2,134.1000.02 Столовая ХБ Оплата за отопление по...,М.Амурского 42,8,0
77,2,134.1000.02 Столовая ХБ Оплата за отопление по...,М.Амурского 42,9,0
78,2,134.1000.02 Столовая ХБ Оплата за отопление по...,М.Амурского 42,10,0
79,2,134.1000.02 Столовая ХБ Оплата за отопление по...,М.Амурского 42,11,0


In [393]:
input_file.drop(columns=['definition'])

,def,street,year,given
1,1,Океанский 34,2,0
2,1,Океанский 34,3,0
3,1,Океанский 34,4,0
4,1,Океанский 34,5,0
5,1,Океанский 34,6,0
...,...,...,...,...
76,2,М.Амурского 42,8,0
77,2,М.Амурского 42,9,0
78,2,М.Амурского 42,10,0
79,2,М.Амурского 42,11,0


In [395]:
to_pred = (input_file
       .assign(n=input_file.groupby(['street', 'year']).cumcount())
       .pivot_table(columns=['date'], values=['given'], index=['street', 'year', 'def'], fill_value=0)
       )

to_pred.to_csv('to_pred.csv')

KeyError: 'date'

In [365]:
to_pred

given                                                  \
date                      1          2              3         4           5    
street year def                                                                
1.0    2016 5.0        0.000   21330.22   24348.710000  21696.45  25386.7800   
            6.0        0.000   54133.09   47974.530000  40132.41  26865.6300   
            7.0      538.230     708.20     736.526667    358.96    882.8175   
       2017 5.0     8141.060   30356.60    5968.230000  13546.67  26469.2800   
            6.0    51325.470   52161.40   53143.920000  39668.11  31544.6300   
...                      ...        ...            ...       ...         ...   
132.0  2021 5.0    87810.720   83238.14   81613.100000  85691.08  85958.6600   
            9.0   169220.040  127554.81  105461.900000  87884.96  54349.8700   
            10.0     536.976     455.78     455.780000    455.78    455.7800   
133.0  2021 9.0        0.000    6292.82    6292.820000   6292.82      0.0000   
134.0  2019 8.0      417.700      55.72       0.000000     43.16      0.0000   

                                                                          \
date                        6             7             8             9    
street year def                                                            
1.0    2016 5.0   22238.080000   8115.533333  18901.520000  27978.350000   
            6.0    5259.580000  -3482.250000      0.000000      0.000000   
            7.0     963.146667    -15.072500   1335.146667   1243.923333   
       2017 5.0   26992.810000  24477.850000  34989.730000  29371.930000   
            6.0    4511.850000      0.000000      0.000000      0.000000   
...                        ...           ...           ...           ...   
132.0  2021 5.0   89589.320000  65937.980000  69230.760000  70241.770000   
            9.0   27601.665000    712.940000    748.330000  -5897.460000   
            10.0    455.780000    455.780000    512.495000   1917.695000   
133.0  2021 9.0       0.000000      0.000000      0.000000      0.000000   
134.0  2019 8.0       0.000000      0.000000      0.000000      0.000000   

                                                              
date                        10             11             12  
street year def                                               
1.0    2016 5.0   24070.430000   26939.060000   32267.665000  
            6.0       0.000000       0.000000   50795.095000  
            7.0     733.596667     861.173333     813.251667  
       2017 5.0   28638.720000   32199.540000   15747.605000  
            6.0       0.000000    5158.260000   23606.515000  
...                        ...            ...            ...  
132.0  2021 5.0   71432.230000   81697.990000   92885.100000  
            9.0   25790.070000  127876.280000  206253.040000  
            10.0    591.106667     591.106667     591.106667  
133.0  2021 9.0       0.000000       0.000000       0.000000  
134.0  2019 8.0       0.000000       0.000000       0.000000  

[662 rows x 12 columns]

# развертка данных

In [362]:
input_file = pd.DataFrame(pd.read_csv('/kaggle/working/fff.csv'))
input_file

,Unnamed: 0,Unnamed: 1,Unnamed: 2,given,given.1,given.2,given.3,given.4,given.5,given.6,given.7,given.8,given.9,given.10,given.11
0,date,NaN,NaN,1.000,2.00,3.000000,4.00,5.0000,6.000000,7.000000,8.000000,9.000000,10.000000,11.000000,12.000000
1,street,year,def,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,2016,5.0,0.000,21330.22,24348.710000,21696.45,25386.7800,22238.080000,8115.533333,18901.520000,27978.350000,24070.430000,26939.060000,32267.665000
3,1.0,2016,6.0,0.000,54133.09,47974.530000,40132.41,26865.6300,5259.580000,-3482.250000,0.000000,0.000000,0.000000,0.000000,50795.095000
4,1.0,2016,7.0,538.230,708.20,736.526667,358.96,882.8175,963.146667,-15.072500,1335.146667,1243.923333,733.596667,861.173333,813.251667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
659,132.0,2021,5.0,87810.720,83238.14,81613.100000,85691.08,85958.6600,89589.320000,65937.980000,69230.760000,70241.770000,71432.230000,81697.990000,92885.100000
660,132.0,2021,9.0,169220.040,127554.81,105461.900000,87884.96,54349.8700,27601.665000,712.940000,748.330000,-5897.460000,25790.070000,127876.280000,206253.040000
661,132.0,2021,10.0,536.976,455.78,455.780000,455.78,455.7800,455.780000,455.780000,512.495000,1917.695000,591.106667,591.106667,591.106667
662,133.0,2021,9.0,0.000,6292.82,6292.820000,6292.82,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
